In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Some necessary imported softwares...
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose,STL
from sklearn.metrics import mean_squared_error , mean_absolute_error
from statsmodels.tools.eval_measures import rmse
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#Read your datasets!

covid_india = pd.read_csv('../input/covid19-india/case_time_series.csv')
india_power_consumption = pd.read_csv('../input/state-wise-power-consumption-in-india/dataset_tk.csv')
bitcoin_price = pd.read_csv('../input/bitcoin-time-series-with-different-time-intervals/BTC-USD-1-DAY.csv')
amazon_stock = pd.read_csv('../input/amazon-stock-price-1997-to-2020/Amazon.csv')
mongolia_meat = pd.read_csv('../input/ub-meat-prices/meat_prices_20180103_20211027.csv')


In [4]:
#Light Data Cleaning, organizing the meats. Deleted first day from
# Beef and Mutton so that all data subsets were even.
mongolia_meat = mongolia_meat.sort_values(by=['meat_type','date'])
mongolian_beef = mongolia_meat.iloc[1:199]
mongolian_goat = mongolia_meat.iloc[200:398]
mongolian_horse = mongolia_meat.iloc[399:597]
mongolian_mutton = mongolia_meat.iloc[598:]
print(f'The lengths are: {len(mongolian_beef), len(mongolian_goat) , len(mongolian_horse) , len(mongolian_mutton)}')

In [5]:
mongolian_beef

In [6]:
# The last 22 entries are the 22 entries from June 2nd to October 27th, in 2021. These will be the dates
# that will be predicted on.
mongolian_mutton[-22:]

In [7]:
#Reset the dates to MM/DD/YEAR
for df in [mongolian_beef , mongolian_goat, mongolian_horse , mongolian_mutton]:
    df.date = df.date.apply(lambda date: date[5:] + "-" + date[:4])

In [8]:
#Now dates are updated:
mongolian_beef

In [9]:
#Now, remove the meat type, fix up the column indices, and make the date the index.
for df in [mongolian_beef , mongolian_goat, mongolian_horse , mongolian_mutton]:
    df.drop('meat_type',axis = 'columns',inplace = True)
    df.set_axis(['Date' , 'Price'],axis = 'columns',inplace = True)
    df.Date = pd.to_datetime(df.Date)
    df.set_index('Date',inplace = True)

In [10]:
#Split up data into training and testing data.
beef_train = mongolian_beef[:-22]
beef_test = mongolian_beef[-22:]

goat_train = mongolian_goat[:-22]
goat_test = mongolian_goat[-22:]

horse_train = mongolian_horse[:-22]
horse_test = mongolian_horse[-22:]

mutton_train = mongolian_mutton[:-22]
mutton_test = mongolian_mutton[-22:]


In [11]:
#Import Seaborn
import seaborn as sns

In [12]:
#A Small Peek at the price behavior...
plt.figure(figsize = (24,9))
ax = sns.lineplot(beef_train.index , beef_train.Price)

In [13]:
#Set Frequency manually, baby! The data itself sometimes uses 6 days, so just run with it as is.
#Seasonal Decomposition Below
beef_decomp = seasonal_decompose(beef_train.Price , model = 'additive',period = 7)
beef_decomp.plot();

In [14]:
#Use SARIMA to forecast the prices! # After switching to 612 3039, results changed.

arima_beef = SARIMAX(beef_train.Price ,  order = (6,1,2) , seasonal_order = (3,0,3,9))
arima_beef_result = arima_beef.fit()
arima_beef_result.summary()

In [15]:
#Predict on Beef.
arima_beef_pred = arima_beef_result.predict(start = len(beef_train) , end = len(mongolian_beef) - 1 , typ = 'levels').rename('ARIMA BEEF')
total_predictions = pd.DataFrame()
total_predictions["ARIMA BEEF"] = arima_beef_pred
arima_beef_pred

In [16]:
# Beef Results for ARIMA Prediction
plt.figure(figsize = (16,9))
plt.plot_date(beef_test.index , beef_test.Price , linestyle = '-')
plt.plot_date(beef_test.index , arima_beef_pred , linestyle = '-.')
plt.legend(labels = ['Test Data' , 'ARIMA Prediction'])
plt.show()
#This was really quite lackluster. I am not sure why it predicted this poorly.
#It may be the amount of data predicted on, it might just be the beef, it might
#just be the latter part of beef having high variability. I will compare
#the results when I do the other meats.
#This modified orders result is a bit better. Let's try it on the other meats.

In [17]:
#Calculate MSE, MAE for Beef Arima forecast.
beef_arima_mse = mean_squared_error(beef_test.Price , arima_beef_pred , squared = True)
beef_arima_mae = mean_absolute_error(beef_test.Price , arima_beef_pred)
print(f'The MSE and MAE when using ARIMA to predict beef were {beef_arima_mse} and {beef_arima_mae}.')

In [18]:
#Now, repeat the process for goat, horse, and mutton meat.
plt.figure(figsize = (16,9))
sns.lineplot(goat_train.index , goat_train.Price);

In [19]:
#Decompose the time series using statsmodels!
goat_decompose = seasonal_decompose(goat_train.Price , model = 'additive' , period = 7)
goat_decompose.plot();

In [20]:
#Apply ARIMA model to the goat time series.
arima_goat = SARIMAX(goat_train.Price , order = (6,1,2) , seasonal_order = (3,0,3,9))
arima_goat_result = arima_goat.fit()
arima_goat_result.summary()

In [21]:
#Predict with the model on goat.
arima_goat_pred = arima_goat_result.predict(start = len(goat_train), end = len(mongolian_goat) - 1,typ = 'levels').rename("ARIMA GOAT")
total_predictions["ARIMA GOAT"] = arima_goat_pred
arima_goat_pred

In [22]:
#Compare goat forecast via lineplot.
plt.figure(figsize = (16,9))
plt.plot_date(goat_test.index , goat_test.Price,linestyle = '-')
plt.plot_date(goat_test.index,arima_goat_pred,linestyle = '-.')
plt.legend(labels = ['Test Data','ARIMA Prediction'])
plt.show()

In [23]:
#Give the MSE and MAE for the goat forecast. Modified order was better, still not great.
goat_arima_mse = mean_squared_error(goat_test.Price , arima_goat_pred , squared = True)
goat_arima_mae = mean_absolute_error(goat_test.Price , arima_goat_pred)
print(f'The MSE and MAE when using ARIMA to predict goat were {goat_arima_mse} and. {goat_arima_mae}.')

In [24]:
#From this point on, I will not be giving notes for the horse and mutton. Process is the same.
#Give a seaborn lineplot,decompose the signal, apply ARIMA, plot date both, and give MSE/MAE.
plt.figure(figsize = (24,14))
sns.lineplot(horse_train.index , horse_train.Price);

In [25]:
horse_decompose = seasonal_decompose(horse_train.Price ,model = 'additive',period = 7)
horse_decompose.plot();

In [26]:
arima_horse = SARIMAX(horse_train.Price , order = (6,1,2) , seasonal_order = (3,0,3,9))
arima_horse_result = arima_horse.fit()
arima_horse_result.summary()

In [27]:
arima_horse_pred = arima_horse_result.predict(start = len(horse_train),end = len(mongolian_horse) - 1 , typ = 'levels').rename("ARIMA HORSE")
total_predictions["ARIMA Horse"] = arima_horse_pred
arima_horse_pred

In [28]:
plt.figure(figsize = (16,9))
plt.plot_date(horse_test.index , horse_test.Price , linestyle = '-')
plt.plot_date(horse_test.index , arima_horse_pred , linestyle = '-.')
plt.legend(labels = ['Test Data' , 'ARIMA Prediction'])
plt.show();

In [29]:
#Modified order results on horse didn't do much of anything.
horse_arima_mse = mean_squared_error(horse_test.Price , arima_horse_pred , squared = True)
horse_arima_mae = mean_absolute_error(horse_test.Price , arima_horse_pred)
print(f'The MSE and MAE when using ARIMA to predict horse were {horse_arima_mse} and {horse_arima_mae}')

In [30]:
#These are getting noticibly worse... At first it only slightly bothered me, and
#now I'm fairly confused and fairly bothered. Onto mutton!
plt.figure(figsize = (16,9))
sns.lineplot(mutton_train.index , mutton_train.Price);

In [31]:
mutton_decompose = seasonal_decompose(mutton_train.Price , model = 'additive' , period = 7)
mutton_decompose.plot();

In [32]:
arima_mutton = SARIMAX(mutton_train.Price ,  order = (6,1,2) , seasonal_order = (3,0,3,9))
arima_mutton_result = arima_mutton.fit()
arima_mutton_result.summary()

In [33]:
arima_mutton_pred = arima_mutton_result.predict(start = len(mutton_train) , end = len(mongolian_mutton) - 1,typ = 'levels').rename("ARIMA Mutton")
total_predictions["ARIMA Mutton"] = arima_mutton_pred
arima_mutton_pred

In [34]:
plt.figure(figsize = (24,15))
plt.plot_date(mutton_test.index , mutton_test.Price,linestyle = '-')
plt.plot_date(mutton_test.index , arima_mutton_pred , linestyle = '-.')
plt.legend(labels = ['Test Data', 'ARIMA Prediction'])
plt.show()

In [35]:
#Modified orders on mutton didn't do much either. Orders most likely have to be modified 
# per every single time series.
arima_mutton_mse = mean_squared_error(mutton_test.Price , arima_mutton_pred , squared = True)
arima_mutton_mae = mean_absolute_error(mutton_test.Price , arima_mutton_pred)
print(f'The MSE and MAE from ARIMA to predict mutton were {arima_mutton_mse} and {arima_mutton_mae}')

In [36]:
#I wonder.....

toy_beef = beef_train.copy()
toy_test = beef_test.copy()
toy_arima_beef = SARIMAX(toy_beef.Price , order = (6,1,2) , seasonal_order = (3,0,3,9))
# 6 1 2.   3 0 3 9
toy_ab = toy_arima_beef.fit()
toy_ab_pred = toy_ab.predict(start = len(toy_beef) , end = len(mongolian_beef)-1 , typ = 'levels')
toy_ab_pred

In [37]:
plt.figure(figsize = (24,15))
plt.plot_date(beef_test.index , toy_test.Price , linestyle = '-')
plt.plot_date(beef_test.index , toy_ab_pred , linestyle = '-.')
plt.legend(labels = ['Test Data','ARIMA Prediction'])
plt.show()

In [38]:
#Now, use LSTM to forecast! First, scale all the data using SKlearn's MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

beef_scaler = MinMaxScaler()
beef_scaler.fit(beef_train)
s_beef_train = beef_scaler.transform(beef_train)
s_beef_test = beef_scaler.transform(beef_test)

In [39]:
#Create a Time Series Object.

from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 22
n_features = 1
beef_gen = TimeseriesGenerator(s_beef_train , s_beef_train,length = n_input , batch_size = 1)

In [40]:
#Import the necessary software, then create your neural network. Use MSE, then MAE.
from keras.models import Sequential
from keras.layers import LSTM , Dense

beef_lstm_mse = Sequential()
beef_lstm_mse.add(LSTM(400,activation = 'relu',input_shape = (n_input,n_features)))
beef_lstm_mse.add(Dense(1))
beef_lstm_mse.compile(optimizer = 'adam', loss = 'mse')

beef_lstm_mse.summary()

In [41]:
beef_lstm_mae = Sequential()
beef_lstm_mae.add(LSTM(400,activation = 'relu',input_shape = (n_input,n_features)))
beef_lstm_mae.add(Dense(1))
beef_lstm_mae.compile(optimizer = 'adam', loss = 'mae')

beef_lstm_mae.summary()

In [42]:
#Fit both generators.

beef_lstm_mse.fit_generator(beef_gen,epochs = 20)

In [43]:
beef_lstm_mae.fit_generator(beef_gen,epochs = 20)

In [44]:
#Plot the MSE and MAE losses against the epochs.
loss_beef_mse = beef_lstm_mse.history.history['loss']
loss_beef_mae = beef_lstm_mae.history.history['loss']
plt.figure(figsize = (18,10))
plt.xticks(np.arange(1,22,1))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(range(len(loss_beef_mae)),loss_beef_mse,'g^')
plt.plot(range(len(loss_beef_mse)),loss_beef_mae,'bo')
plt.legend(labels = ['MSE Loss','MAE Loss'])
plt.show()

In [45]:
#Predict using MSE model then MAE model.

mse_beef_predictions_scaled = list()
beef_batch = s_beef_train[-n_input:]
beef_current_batch = beef_batch.reshape((1,n_input,n_features))

for j in range(len(s_beef_test)):
    beef_lstm_mse_pred = beef_lstm_mse.predict(beef_current_batch)[0]
    mse_beef_predictions_scaled.append(beef_lstm_mse_pred)
    beef_current_batch = np.append(beef_current_batch[:,1:,:],[[beef_lstm_mse_pred]],axis = 1)

mse_beef_predictions = beef_scaler.inverse_transform(mse_beef_predictions_scaled)
total_predictions["LSTM MSE Beef"] = mse_beef_predictions

In [46]:
#MAE Model predictions 

mae_beef_predictions_scaled = list()
beef_batch = s_beef_train[-n_input:]
beef_current_batch = beef_batch.reshape((1,n_input,n_features))

for j in range(len(s_beef_test)):
    beef_lstm_mae_pred = beef_lstm_mae.predict(beef_current_batch)[0]
    mae_beef_predictions_scaled.append(beef_lstm_mae_pred)
    beef_current_batch = np.append(beef_current_batch[:,1:,:],[[beef_lstm_mae_pred]],axis = 1)

mae_beef_predictions = beef_scaler.inverse_transform(mae_beef_predictions_scaled)
total_predictions["LSTM MAE Beef"] = mae_beef_predictions

In [47]:
#Plot the MSE and MAE predictions against the real thing.
plt.figure(figsize = (20,12))
plt.plot_date(beef_test.index , beef_test.Price , linestyle = '-')
plt.plot_date(beef_test.index , mse_beef_predictions , linestyle = '-.')
plt.plot_date(beef_test.index,  mae_beef_predictions,linestyle = '--')
plt.legend(labels = ['Test Data','LSTM MSE Prediction','LSTM MAE Prediction'])
plt.show();

In [48]:
#Repeat this process using LSTM with Goat, Horse, and Mutton. The process is as follows:
# Scale the data, create a time series object. Create the NN using MSE, MAE.
# Fit the generators accordingly.
# Plot the MSE, MAE losses against epochs.
# Plot the predictions against the real thing.
# For Goat, I will use 352 nodes, and see what happens.

In [49]:
goat_scaler = MinMaxScaler()
goat_scaler.fit(goat_train)
s_goat_train = goat_scaler.transform(goat_train)
s_goat_test = goat_scaler.transform(goat_test)

In [50]:
n_input = 22
n_features = 1
goat_gen = TimeseriesGenerator(s_goat_train , s_goat_train , length = n_input , batch_size = 1)

In [51]:
goat_lstm_mse = Sequential()
goat_lstm_mse.add(LSTM(352,activation = 'relu',input_shape = (n_input,n_features)))
goat_lstm_mse.add(Dense(1))
goat_lstm_mse.compile(optimizer = 'adam', loss = 'mse')
goat_lstm_mse.summary()

In [52]:
goat_lstm_mae = Sequential()
goat_lstm_mae.add(LSTM(352,activation = 'relu',input_shape = (n_input,n_features)))
goat_lstm_mae.add(Dense(1))
goat_lstm_mae.compile(optimizer = 'adam',loss = 'mae')
goat_lstm_mae.summary()

In [53]:
goat_lstm_mse.fit_generator(goat_gen , epochs = 20)

In [54]:
goat_lstm_mae.fit_generator(goat_gen,epochs = 20)

In [55]:
loss_goat_mse = goat_lstm_mse.history.history['loss']
loss_goat_mae = goat_lstm_mae.history.history['loss']
plt.figure(figsize = (18,10))
plt.xticks(np.arange(1,22,1))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(range(len(loss_goat_mse)),loss_goat_mse , 'r^')
plt.plot(range(len(loss_goat_mae)),loss_goat_mae,'bo')
plt.legend(labels = ['MSE Loss', 'MAE Loss'])
plt.show()

In [56]:
mse_goat_predictions_scaled = list()
goat_batch = s_goat_train[-n_input:]
goat_current_batch = goat_batch.reshape((1,n_input,n_features))

for j in range(len(s_goat_test)):
    goat_lstm_mse_pred = goat_lstm_mse.predict(goat_current_batch)[0]
    mse_goat_predictions_scaled.append(goat_lstm_mse_pred)
    goat_current_batch = np.append(goat_current_batch[:,1:,:],[[goat_lstm_mse_pred]],axis = 1)
    
mse_goat_predictions = goat_scaler.inverse_transform(mse_goat_predictions_scaled)
total_predictions["LSTM MSE Goat"] = mse_goat_predictions


mae_goat_predictions_scaled = list()
goat_batch = s_goat_train[-n_input:]
goat_current_batch = goat_batch.reshape((1,n_input,n_features))

for j in range(len(s_goat_test)):
    goat_lstm_mae_pred = goat_lstm_mae.predict(goat_current_batch)[0]
    mae_goat_predictions_scaled.append(goat_lstm_mae_pred)
    goat_current_batch = np.append(goat_current_batch[:,1:,:],[[goat_lstm_mae_pred]],axis = 1)
    
mae_goat_predictions = goat_scaler.inverse_transform(mae_goat_predictions_scaled)
total_predictions["LSTM MAE Goat"] = mae_goat_predictions

In [57]:
plt.figure(figsize = (20,12))
plt.plot_date(goat_test.index , goat_test.Price , linestyle = '-')
plt.plot_date(goat_test.index , mse_goat_predictions , linestyle = '-.')
plt.plot_date(goat_test.index,  mae_goat_predictions,linestyle = '--')
plt.legend(labels = ['Test Data','LSTM MSE Prediction','LSTM MAE Prediction'])
plt.show();

In [58]:
horse_scaler = MinMaxScaler()
horse_scaler.fit(horse_train)
s_horse_train = horse_scaler.transform(horse_train)
s_horse_test = horse_scaler.transform(horse_test)

In [59]:
n_input = 22
n_features = 1
horse_gen = TimeseriesGenerator ( s_horse_train , s_horse_train , length = n_input , batch_size = 1)

In [60]:
horse_lstm_mae = Sequential()
horse_lstm_mae.add(LSTM(200,activation = 'relu',input_shape = (n_input , n_features)))
horse_lstm_mae.add(Dense(1))
horse_lstm_mae.compile(optimizer = 'adam', loss = 'mae')
horse_lstm_mae.summary()

In [61]:
horse_lstm_mse = Sequential()
horse_lstm_mse.add(LSTM(200 , activation = 'relu', input_shape = (n_input, n_features)))
horse_lstm_mse.add(Dense(1))
horse_lstm_mse.compile(optimizer = 'adam',loss = 'mse')
horse_lstm_mse.summary()

In [62]:
horse_lstm_mse.fit_generator(horse_gen , epochs = 20)

In [63]:
horse_lstm_mae.fit_generator(horse_gen , epochs = 20)

In [64]:
loss_horse_mse = horse_lstm_mse.history.history['loss']
loss_horse_mae = horse_lstm_mae.history.history['loss']
plt.figure(figsize = (18,10))
plt.xticks(np.arange(1,22,1))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(range(len(loss_horse_mse)),loss_horse_mse,'r*')
plt.plot(range(len(loss_horse_mae)),loss_horse_mae,'y^')
plt.legend(labels = ['MSE Loss', 'MAE Loss'])
plt.show();

In [65]:
mse_horse_predictions_scaled = list()
horse_batch = s_horse_train[-n_input:]
horse_current_batch = horse_batch.reshape((1,n_input,n_features))

for j in range(len(s_horse_test)):
    horse_lstm_mse_pred = horse_lstm_mse.predict(horse_current_batch)[0]
    mse_horse_predictions_scaled.append(horse_lstm_mse_pred)
    horse_current_batch = np.append(horse_current_batch[:,1:,:],[[horse_lstm_mse_pred]],axis = 1)
    
mse_horse_predictions = horse_scaler.inverse_transform(mse_horse_predictions_scaled)
total_predictions['LSTM MSE Horse'] = mse_horse_predictions

mae_horse_predictions_scaled = list()
horse_batch = s_horse_train[-n_input:]
horse_current_batch = horse_batch.reshape((1,n_input,n_features))

for j in range(len(s_horse_test)):
    horse_lstm_mae_pred = horse_lstm_mae.predict(horse_current_batch)[0]
    mae_horse_predictions_scaled.append(horse_lstm_mae_pred)
    horse_current_batch = np.append(horse_current_batch[:,1:,:],[[horse_lstm_mae_pred]],axis = 1)
    
mae_horse_predictions = horse_scaler.inverse_transform(mae_horse_predictions_scaled)
total_predictions['LSTM MAE Horse'] = mae_horse_predictions

In [66]:
plt.figure(figsize = (20,12))
plt.plot_date(horse_test.index , horse_test.Price , linestyle = '-')
plt.plot_date(horse_test.index , mse_horse_predictions , linestyle = '-.')
plt.plot_date(horse_test.index,  mae_horse_predictions,linestyle = '--')
plt.legend(labels = ['Test Data','LSTM MSE Prediction','LSTM MAE Prediction'])
plt.show();

In [68]:
#Predict on Mutton
mutton_scaler = MinMaxScaler()
mutton_scaler.fit(mutton_train)
s_mutton_train = mutton_scaler.transform(mutton_train)
s_mutton_test = horse_scaler.transform(mutton_test)

In [69]:
n_input = 22
n_features = 1
mutton_gen = TimeseriesGenerator(s_mutton_train , s_mutton_train , length = n_input , batch_size = 1)

In [70]:
mutton_lstm_mae = Sequential()
mutton_lstm_mae.add(LSTM(400,activation = 'relu',input_shape = (n_input,n_features)))
mutton_lstm_mae.add(Dense(1))
mutton_lstm_mae.compile(optimizer = 'adamax' , loss = 'mae')
mutton_lstm_mae.summary()

In [73]:
mutton_lstm_mse = Sequential()
mutton_lstm_mse.add(LSTM(400,activation = 'relu',input_shape = (n_input , n_features)))
mutton_lstm_mse.add(Dense(1))
mutton_lstm_mse.compile(optimizer = 'adamax',loss = 'mse')
mutton_lstm_mse.summary()

In [72]:
mutton_lstm_mae.fit_generator(mutton_gen , epochs = 20)

In [74]:
mutton_lstm_mse.fit_generator(mutton_gen , epochs = 20)

In [76]:
loss_mutton_mae = mutton_lstm_mae.history.history['loss']
loss_mutton_mse = mutton_lstm_mse.history.history['loss']
plt.figure(figsize = (18,10))
plt.xticks(np.arange(1,22,1))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(range(len(loss_mutton_mae)),loss_mutton_mae , 'g^')
plt.plot(range(len(loss_mutton_mse)),loss_mutton_mse , 'bo')
plt.legend(labels = ['MAE Loss','MSE Loss'])
plt.show();

In [79]:
mae_mutton_predictions_scaled = list()
mutton_batch = s_mutton_train[-n_input:]
mutton_current_batch = mutton_batch.reshape((1,n_input,n_features))

for j in range(len(s_mutton_test)):
    mutton_lstm_mae_pred = mutton_lstm_mae.predict(mutton_current_batch)[0]
    mae_mutton_predictions_scaled.append(mutton_lstm_mae_pred)
    mutton_current_batch = np.append(mutton_current_batch[:,1:,:],[[mutton_lstm_mae_pred]],axis = 1)
    
mae_mutton_predictions = mutton_scaler.inverse_transform(mae_mutton_predictions_scaled)
total_predictions['LTSM MAE Mutton'] = mae_mutton_predictions

mse_mutton_predictions_scaled = list()
mutton_batch = s_mutton_train[-n_input:]
mutton_current_batch = mutton_batch.reshape((1,n_input,n_features))

for j in range(len(s_mutton_test)):
    mutton_lstm_mse_pred = mutton_lstm_mse.predict(mutton_current_batch)[0]
    mse_mutton_predictions_scaled.append(mutton_lstm_mse_pred)
    mutton_current_batch = np.append(mutton_current_batch[:,1:,:],[[mutton_lstm_mse_pred]],axis = 1)
    
mse_mutton_predictions = mutton_scaler.inverse_transform(mse_mutton_predictions_scaled)
total_predictions['LSTM MSE Mutton'] = mse_mutton_predictions

In [80]:
plt.figure(figsize = (20,12))
plt.plot_date(mutton_test.index , mutton_test.Price , linestyle = '-')
plt.plot_date(mutton_test.index , mae_mutton_predictions , linestyle = '-.')
plt.plot_date(mutton_test.index , mse_mutton_predictions , linestyle = '--')
plt.legend(labels = ['Test Data','LSTM MAE Prediction','LSTM MSE Prediction'])
plt.show();

In [87]:
#Now I will forecast on the Mongolian Meat Prices using Prophet.
#First copy all the dataframes, then reset the indices and columns.

df_beef = mongolian_beef.copy()
df_goat = mongolian_goat.copy()
df_horse = mongolian_horse.copy()
df_mutton = mongolian_mutton.copy()

df_beef = df_beef.reset_index()
df_goat = df_goat.reset_index()
df_horse = df_horse.reset_index()
df_mutton = df_mutton.reset_index()

df_beef.columns = ['ds','y']
df_goat.columns = ['ds','y']
df_horse.columns = ['ds','y']
df_mutton.columns = ['ds','y']

In [88]:
#Create the train and test data.

pr_beef_train = df_beef.iloc[:-22]
pr_beef_test = df_beef.iloc[-22:]

pr_goat_train = df_goat.iloc[:-22]
pr_goat_test = df_goat.iloc[-22:]

pr_horse_train = df_horse.iloc[:-22]
pr_horse_test = df_horse.iloc[-22:]

pr_mutton_train = df_mutton.iloc[:-22]
pr_mutton_test = df_mutton.iloc[-22:]

In [89]:
from prophet import Prophet

In [90]:
#Instantiate the model.
pr_beef_model = Prophet()
pr_goat_model = Prophet()
pr_horse_model = Prophet()
pr_mutton_model = Prophet()

In [91]:
#Fit the model.
pr_beef_model.fit(pr_beef_train)
pr_goat_model.fit(pr_goat_train)
pr_horse_model.fit(pr_horse_train)
pr_mutton_model.fit(pr_mutton_train)

In [94]:
#Make the DataFrames.
pr_beef_future = pr_beef_model.make_future_dataframe(periods = 12 , freq = 'MS')
pr_goat_future = pr_goat_model.make_future_dataframe(periods = 12 , freq = 'MS')
pr_horse_future = pr_horse_model.make_future_dataframe(periods = 12 , freq = 'MS')
pr_mutton_future = pr_mutton_model.make_future_dataframe(periods = 12 , freq = 'MS')

In [96]:
#Predict on the Data.
pr_beef_pred = pr_beef_model.predict(pr_beef_future)
pr_goat_pred = pr_goat_model.predict(pr_goat_future)
pr_horse_pred = pr_horse_model.predict(pr_horse_future)
pr_mutton_pred = pr_mutton_model.predict(pr_mutton_future)

In [97]:
#Restructure the data.
pr_beef_pred = pd.DataFrame({"Date": pr_beef_pred[-22:]['ds'] , "Pred": pr_beef_pred[-22:]['yhat']})
pr_goat_pred = pd.DataFrame({"Date": pr_goat_pred[-22:]['ds'] , "Pred": pr_goat_pred[-22:]['yhat']})
pr_horse_pred = pd.DataFrame({"Date": pr_horse_pred[-22:]['ds'] , "Pred": pr_horse_pred[-22:]['yhat']})
pr_mutton_pred = pd.DataFrame({"Date": pr_mutton_pred[-22:]['ds'] , "Pred": pr_mutton_pred[-22:]['yhat']})

In [98]:
#Reset indices.
pr_beef_pred = pr_beef_pred.set_index("Date")
pr_goat_pred = pr_goat_pred.set_index("Date")
pr_horse_pred = pr_horse_pred.set_index("Date")
pr_mutton_pred = pr_mutton_pred.set_index("Date")

In [101]:
#Add the predicted values to the total predictions df.

total_predictions['Prophet Beef'] = pr_beef_pred['Pred'].values
total_predictions['Prophet Goat'] = pr_goat_pred['Pred'].values
total_predictions['Prophet Horse'] = pr_horse_pred['Pred'].values
total_predictions['Prophet Mutton'] = pr_mutton_pred['Pred'].values

In [125]:
import seaborn as sns

plt.figure(figsize = (9,5))
f1 = sns.lineplot(x = pr_beef_test.index , y = pr_beef_pred['Pred'])
sns.lineplot(x = pr_beef_test.index , y = pr_beef_test['y'])

In [109]:
plt.figure(figsize = (9,5))
f2 = sns.lineplot(x = pr_goat_test.index , y = pr_goat_pred['Pred'])
sns.lineplot(x = pr_goat_test.index , y = pr_goat_test['y'])

In [110]:
plt.figure(figsize = (9,5))
f3 = sns.lineplot(x = pr_horse_test.index , y = pr_horse_pred['Pred'])
sns.lineplot(x = pr_horse_test.index , y = pr_horse_test['y'])

In [111]:
plt.figure(figsize = (9,5))
f4 = sns.lineplot(x = pr_mutton_test.index , y = pr_mutton_pred['Pred'])
sns.lineplot(x = pr_mutton_test.index , y = pr_mutton_test['y'])

In [121]:
pr_beef_mse = rmse(pr_beef_test['y'] , pr_beef_pred['Pred']) ** 2
pr_goat_mse = rmse(pr_goat_test['y'],pr_goat_pred['Pred']) ** 2
pr_horse_mse = rmse(pr_horse_test['y'] , pr_horse_pred['Pred'])**2
pr_mutton_mse = rmse(pr_mutton_test['y'],pr_mutton_pred['Pred'])**2

MSE = [pr_beef_mse , pr_goat_mse , pr_horse_mse , pr_mutton_mse]
meat_names = ['beef', 'goat','horse','mutton']

for j in range(4):
    print(f'The MSE using Prophet for ' + meat_names[j] + f' was {MSE[j]}.')

In [ ]:
#What have I learned? There's something fishy going on with these models. I haven't quite wrangled with 
#what's going on, and I want to figure out how I can get better forecasting results. I think I need to look into
# optimizing based on parameters. Firstly, I want to figure out what dataset to use next.

In [133]:
amazon_stock.head(n=5)
#There seems to be some missing dates here... this data won't work all that well with frequency. What about BTC?

In [131]:
bitcoin_price.head(n=40)

In [143]:
dates = bitcoin_price['Date']
dates.tail(n=30)
#Taking into account 2020 was a leap year, all of 2020 and January - September is 639 days.
#I could predict on October, but some of October is missing. At this point, I think it most applicable that I 
#should use all of 2020 and Jan-May to train, and then predict on June, one month thereafter.